<a href="https://colab.research.google.com/github/duartemiguel30/colab_tia/blob/main/C%C3%B3digo_funcional_novo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers datasets accelerate peft bitsandbytes trl
from huggingface_hub import login
login()

In [ ]:
from datasets import load_dataset, ClassLabel
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

train_path = "/content/casos_convertidos_com_invalidos.jsonl"
dataset = load_dataset("json", data_files=train_path, split="train")

labels = ["amarela", "laranja", "caso inválido", "vermelha", "azul", "verde"]
class_label = ClassLabel(names=labels)

def converter_label(ex):
    ex["label_id"] = class_label.str2int(ex["labels"])
    return ex

dataset = dataset.map(converter_label)

modelo_nome = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(modelo_nome)

def tokenizar(ex):
    return tokenizer(
        ex["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

dataset = dataset.map(tokenizar)
dataset = dataset.remove_columns(["labels"])
dataset = dataset.rename_column("label_id", "labels")
dataset = dataset.remove_columns(["text"])

modelo = AutoModelForSequenceClassification.from_pretrained(
    modelo_nome,
    num_labels=len(labels),
    id2label={i: l for i, l in enumerate(labels)},
    label2id={l: i for i, l in enumerate(labels)}
)

args = TrainingArguments(
    output_dir="./modelo_treinado",
    per_device_train_batch_size=16,
    num_train_epochs=2,
    learning_rate=2e-5,
    logging_dir="./logs",
    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=modelo,
    args=args,
    train_dataset=dataset,
    tokenizer=tokenizer
)

trainer.train()
modelo.save_pretrained("./modelo_treinado")
tokenizer.save_pretrained("./modelo_treinado")


In [ ]:
from transformers import pipeline

modelo_path = "./modelo_treinado"
clf = pipeline("text-classification", model=modelo_path, tokenizer=modelo_path)

texto = "Paciente com dores da cabeça ."
resultado = clf(texto)
print("Resultado:", resultado)
